# **Create Descriptions**

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import torch
from datasets import load_dataset
from transformers import InstructBlipForConditionalGeneration, InstructBlipProcessor

In [ ]:
model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-vicuna-7b",
    load_in_4bit = True,
    torch_dtype = torch.bfloat16,
)

In [ ]:
processor = InstructBlipProcessor.from_pretrained(
    "Salesforce/instructblip-vicuna-7b",
)

In [ ]:
datasets = [
    ("detection-datasets/fashionpedia", None, "val"),
    ("keremberke/nfl-object-detection", "mini", "test"),
    ("keremberke/plane-detection", "mini", "train"),
    ("Matthijs/snacks", None, "validation"),
    ("rokmr/mini_pets", None, "test"),
    ("keremberke/pokemon-classification", "mini", "train"),
]

In [ ]:
prompt1 = "describe this image in full detail. describe each and every aspect of the image so that an artist could re create the image"
prompt2 = "create an extensive description of this image"

In [ ]:
counter = 0
for name, config, split in datasets:
  d = load_dataset(name, config, split = split)
  for idx in range(len(d)):
    image = d[idx]["image"]
    desc = ""
    for _prompt in [prompt1, prompt2]:
      inputs = processor(
          images = image,
          text = _prompt,
          return_rensors = "pt"
      ).to(model.device, torch.bfloat16)
      outputs = model.generate(
          **inputs,
          do_sample = False,
          num_beams = 10,
          max_length = 512,
          min_length = 16,
          top_p = 0.9,
          repetition_penalty = 1.5,
          temperature = 1,
      )
      generated_text = processor.batch_decode(
          outputs,
          skip_special_tokens = True,
      )[0].strip()

      desc += generated_text + " "

    desc = desc.strip() #remove \n \t
    image.save(f"images/{counter}.jpg")
    print(counter, desc)

    with open("description.csv", "a") as f:
      f.write(f"{counter}, {desc}\n")

    counter+=1
    torch.cuda.empty_cache()

# **Create Embeddings**

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import pickle
import pandas as pd

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

with open("descriptions.csv", "r") as f:
  lines = f.readlines()

lines = [line.strip().split(",") for line in lines]

for idx, line in enumerate(lines):
  lines[idx] = [line[0], ",".join(line[1:])]

df = pd.Dataframe(lines, columns = ["id", "desc"])
embeddings = model.encode(df["desc"].tolist(), show_progress_bar = True)

with open("embeddings.pkl", "wb") as f:
  pickle.dump(embeddings, f)

print(embeddings.shape)


# **App**

In [ ]:
import gradio as gr
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTranformer("sentence-transformers/all-mpnet-base-v2")

with open("embeddings.pkl", "rb") as f:
  embeddings = pickle.load(f)
embeddings = embeddings.astype("float32")

embedding_size = embeddings.shape[1]
n_clusters = 5
num_results = 5

quantizer = faiss.IndexFlatIP(embedding_size)
index = faiss.IndexIVFFlat(
    quantizer, embedding_size, n_clusters, faiss.METRIC_INNER_PRODUCT,
)

index.train(embeddings)
index.add(embeddings)

def _search(query):
  query_embedding = model.encode(query)
  query_embedding = np.array(query_embeedding).astype("float32")
  query_embedding = query_embedding.reshape(1, -1)
  _, indices = index.search(query_embedding, num_results)
  images = [f"images/{i}.jpg" for i in indices[0]]
  return images

with gr.Blocks() as demo:
  query = gr.Textbox(lines = 1, label = "search query")
  outputs = gr.Gallery(preview = True)
  submit = gr.Button(value = "search")
  submit.click(_search, inputs = query, outputs = outputs)

demo.launch()